In [9]:
from datasets import load_dataset
import random

In [13]:
DATASET= 'Cameron-Chen/mixed_qa'
DEFAULT_SUBSET = 150

In [6]:
ds = load_dataset(DATASET,split='test')

In [7]:
ds

Dataset({
    features: ['problem', 'answer'],
    num_rows: 11015
})

In [11]:
indices = list(range(len(ds)))
random.shuffle(indices)

In [14]:
n=DEFAULT_SUBSET

In [15]:
indices = indices[:n]
prompts = []
refs = []

In [17]:
len(indices)

150

In [20]:
for i, idx in enumerate(indices[:2]):
    print(i)
    row = ds[int(idx)]

0
1


In [21]:
row

{'problem': 'What day of the week does the Korean cooking-variety program featuring the owner of "Dear. bread" air?',
 'answer': ['Mondays']}

In [ ]:
def prepare_truthful_qa(n: int = DEFAULT_SUBSET, 
                        split: str = "validation", 
                        seed: int = SEED) -> Tuple[str, str]):
    random.seed(seed)
    dataset= "truthful_qa"
    ds = load_dataset(dataset)[split]
    # Each row has 'question' and 'best_answer' (plus more fields); use best_answer as reference.
    indices = list(range(len(ds)))
    random.shuffle(indices)
    indices = indices[:n]
    prompts = []
    refs = []
    for i, idx in enumerate(indices):
        row = ds[int(idx)]
        q = row["question"]
        best = (row.get("best_answer") or "").strip()
        # TruthfulQA sometimes provides additional correct answers
        extra = row.get("correct_answers") or row.get("correct_answers_list") or []
        ref_list = []
        if best:
            ref_list.append(best)
        # ensure strings & strip
        for r in extra:
            if isinstance(r, str):
                r = r.strip()
                if r and r not in ref_list:
                    ref_list.append(r)
        rid = f"truth-{i:04d}"
        prompts.append({
            "id": rid,
            "prompt": q,
            "system": "You are a concise, truthful assistant. Answer briefly and accurately."
        })
        # Store both a list for robust scoring and a single field for back-compat
        refs.append({"id": rid, "references": ref_list, "reference": best})
    prompts_path = os.path.join(DATA_DIR, f"{dataset}_subset.jsonl")
    refs_path = os.path.join(DATA_DIR, f"{dataset}_refs.jsonl")
    write_jsonl(prompts_path, prompts)
    write_jsonl(refs_path, refs)